In [1]:
from importlib.metadata import version

print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

torch version: 2.8.0+cu126
tiktoken version: 0.11.0


In [2]:
import os
import urllib.request

if not os.path.exists("the-verdict.txt"):
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
    file_path = "the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)

In [3]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [4]:
import re
text = raw_text[:99]
result = re.split(r'([,.]|\s)', text)
print(result)

['I', ' ', 'HAD', ' ', 'always', ' ', 'thought', ' ', 'Jack', ' ', 'Gisburn', ' ', 'rather', ' ', 'a', ' ', 'cheap', ' ', 'genius--though', ' ', 'a', ' ', 'good', ' ', 'fellow', ' ', 'enough--so', ' ', 'it', ' ', 'was', ' ', 'no', ' ', '']


In [ ]:
# Strip whitespace from each item and then filter out any empty strings.
result = [item for item in result if item.strip()]
print(result)

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius--though', 'a', 'good', 'fellow', 'enough--so', 'it', 'was', 'no']


In [ ]:
Text = "Hello, world. Is this-- a @ test?"
result = re.split(r'([,.:?_!@#]|--|\s)', Text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', '@', 'test', '?']


In [ ]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [ ]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
print(vocab_size)

1130


In [ ]:
vocab = {token : integar for integar, token in enumerate(all_words)}

for i, item in enumerate(vocab.items()):
  print(item)
  if i >= 50:
    break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


In [ ]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)

        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [ ]:
tokenizer = SimpleTokenizerV1(vocab)

text = """"It's the last he painted, you know,"
           Mrs. Gisburn said with pardonable pride."""
text_modfiy = re.split(r'([,.:;?_!"()\']|--|\s)', text)
text_modfiy = [item.strip() for item in text_modfiy if item.strip()]
ids = tokenizer.encode(text)
text_back = tokenizer.decode(ids)
print(ids), print(text_modfiy), print(text_back)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]
['"', 'It', "'", 's', 'the', 'last', 'he', 'painted', ',', 'you', 'know', ',', '"', 'Mrs', '.', 'Gisburn', 'said', 'with', 'pardonable', 'pride', '.']
" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


(None, None, None)

In [ ]:
tokenizer.decode(ids)


'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [ ]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer,token in enumerate(all_tokens)}

In [ ]:
len(vocab.items())

1132

In [ ]:
for i, item in enumerate(list(vocab.items())[-7:]):
  print(item)


('yet', 1125)
('you', 1126)
('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [ ]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int
            else "<|unk|>" for item in preprocessed
        ]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

In [ ]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join((text1, text2))

print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [ ]:
tokenizer.encode(text)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]

In [ ]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'


**2.5 BytePair encoding**

In [ ]:
pip install tiktoken

In [5]:
import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))

tiktoken version: 0.11.0


In [6]:
tokenizer = tiktoken.get_encoding("gpt2")

In [7]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [ ]:
tokenizer.decode(integers)

'Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.'

In [ ]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [ ]:
enc_sample = enc_text[50:]

In [ ]:
context_size = 4
x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]
print(f"x:", x)
print(f"y:    ", y)

x: [290, 4920, 2241, 287]
y:     [4920, 2241, 287, 257]


In [ ]:
for i in range(1, context_size+1):
  context = enc_sample[:i]
  desired = enc_sample[i]
  print(f"x: {context}, y: {desired}")

x: [290], y: 4920
x: [290, 4920], y: 2241
x: [290, 4920, 2241], y: 287
x: [290, 4920, 2241, 287], y: 257


In [ ]:
for i in range(1, context_size+10):
  context = enc_sample[:i]
  desired = enc_sample[i]
  print(tokenizer.decode(context),"---->", tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a
 and established himself in a ---->  vill
 and established himself in a vill ----> a
 and established himself in a villa ---->  on
 and established himself in a villa on ---->  the
 and established himself in a villa on the ---->  Riv
 and established himself in a villa on the Riv ----> iera
 and established himself in a villa on the Riviera ----> .
 and established himself in a villa on the Riviera. ---->  (
 and established himself in a villa on the Riviera. ( ----> Though


In [8]:
import torch
print("PyTorch version:", torch.__version__)

PyTorch version: 2.8.0+cu126


In [9]:
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
  def __init__(self,txt,tokenizer,max_length,stride):
    self.input_ids = []
    self.target_ids = []
    token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
    assert len(token_ids) > max_length, "Number of tokenized input must at least be equal to max_length+1"
    for i in range(0, len(token_ids)-max_length, stride):
      input_chunk = token_ids[i: i+max_length]
      target_chunk = token_ids[i+1: i+max_length+1]
      self.input_ids.append(torch.tensor(input_chunk))
      self.target_ids.append(torch.tensor(target_chunk))

  def __len__(self):
      return len(self.input_ids)

  def __getitem__(self, idx):
      return self.input_ids[idx], self.target_ids[idx]

In [10]:
dataset = GPTDatasetV1("Hello world! GPT is fun.", tokenizer, max_length=5, stride=1)
input_ids, target_ids = dataset[0]

print("Input:", input_ids)
print("Target:", target_ids)

Input: tensor([15496,   995,     0,   402, 11571])
Target: tensor([  995,     0,   402, 11571,   318])


In [11]:
def create_dataloader_v1(txt, batch_size=4,
                         max_length=256,
                         stride=128,
                         shuffle=True,
                         drop_last=True,
                         num_workers=0):

  tokenizer = tiktoken.get_encoding("gpt2")

  dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

  dataloader = DataLoader(dataset,
                          batch_size=batch_size,
                          shuffle=shuffle,
                          drop_last=drop_last,
                          num_workers=num_workers)
  return dataloader


In [12]:
with open("the-verdict.txt", "r", encoding="utf_8") as f:
  raw_text = f.read()

In [13]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=6, stride=1, shuffle=False
)
data_iter = iter(dataloader)

first_patch = next(data_iter)
print(first_patch)

second_batch = next(data_iter) # get second batch
print(second_batch)

[tensor([[  40,  367, 2885, 1464, 1807, 3619]]), tensor([[ 367, 2885, 1464, 1807, 3619,  402]])]
[tensor([[ 367, 2885, 1464, 1807, 3619,  402]]), tensor([[2885, 1464, 1807, 3619,  402,  271]])]


In [14]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[  286,   262, 33125,    12],
        [ 8208,   286,   465,     0],
        [  340,   438,    62,    40],
        [   11,   355,  9074,    13],
        [    1,    39,   648,   340],
        [12036,   661,   836,   470],
        [  511,   366,  8642,   521],
        [  262,  4324,    12,    66]])

Targets:
 tensor([[  262, 33125,    12,   565],
        [  286,   465,     0,   314],
        [  438,    62,    40,   550],
        [  355,  9074,    13,   536],
        [   39,   648,   340,    11],
        [  661,   836,   470,   910],
        [  366,  8642,   521,   829],
        [ 4324,    12,    66,  3325]])


In [15]:
input_ids = torch.tensor([2,3,4,1])

In [16]:
vocab_size = 6
output_dim = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [17]:
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [18]:
print(embedding_layer(torch.tensor([2])))

tensor([[ 1.2753, -0.2010, -0.1606]], grad_fn=<EmbeddingBackward0>)


In [19]:
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


In [20]:
vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [21]:
max_length = 4
dataloader = create_dataloader_v1(raw_text, batch_size =8, max_length=max_length, stride=max_length, shuffle=False)

data_iter = iter(dataloader)
next_bacth = next(data_iter)
input, target = next_bacth
print(input)
print(input.shape)
print(target)
print(target.shape)

tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
torch.Size([8, 4])
tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])
torch.Size([8, 4])


In [22]:
token_embedding = token_embedding_layer(input)
print(token_embedding.shape)

torch.Size([8, 4, 256])


In [27]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
print(pos_embedding_layer.weight)

Parameter containing:
tensor([[ 0.6021,  1.0476, -0.0039,  ...,  2.0443, -0.7210,  0.2375],
        [ 0.1490, -0.5767, -0.6754,  ...,  1.3159, -0.2923,  0.7291],
        [ 0.7771,  1.5237, -0.2249,  ..., -0.0722, -0.9698,  0.7796],
        [-1.3620,  1.1342, -0.8560,  ...,  0.7717,  0.0025, -0.0927]],
       requires_grad=True)


In [30]:
pos_embedding = pos_embedding_layer(torch.arange(max_length))
print(pos_embedding.shape)
print(pos_embedding)

torch.Size([4, 256])
tensor([[ 0.6021,  1.0476, -0.0039,  ...,  2.0443, -0.7210,  0.2375],
        [ 0.1490, -0.5767, -0.6754,  ...,  1.3159, -0.2923,  0.7291],
        [ 0.7771,  1.5237, -0.2249,  ..., -0.0722, -0.9698,  0.7796],
        [-1.3620,  1.1342, -0.8560,  ...,  0.7717,  0.0025, -0.0927]],
       grad_fn=<EmbeddingBackward0>)


In [32]:
input_embedding = token_embedding + pos_embedding
print(input_embedding.shape)
print(input_embedding)

torch.Size([8, 4, 256])
tensor([[[ 1.0934,  2.1714,  1.4549,  ...,  1.6448, -2.5945,  0.0930],
         [ 0.5970, -0.3231, -0.9410,  ...,  1.8156, -1.4914, -0.4553],
         [ 0.5264,  1.4691,  0.4438,  ...,  0.8895,  1.4038,  0.7268],
         [-0.4163,  2.0000,  0.7631,  ...,  0.3173, -0.7435,  0.2556]],

        [[ 2.1482,  2.7844, -0.7887,  ...,  1.9439,  0.1374, -0.1046],
         [-1.7132, -0.7681, -1.0566,  ...,  2.4379, -0.6419,  1.3382],
         [ 2.7619,  0.8755, -0.3664,  ..., -0.4563, -1.9053,  2.2274],
         [-0.3973,  2.4316, -2.4767,  ...,  1.9180,  1.5821,  0.3042]],

        [[-0.1692,  1.7048,  0.1624,  ...,  1.2400, -0.6668,  0.9801],
         [ 0.9535, -0.0720,  0.6168,  ...,  2.7807,  0.1174,  1.0496],
         [ 0.8566, -0.2399,  0.3501,  ...,  2.1100,  0.8532,  0.4161],
         [-0.9353,  1.0695, -0.2874,  ...,  0.2508,  1.3090,  0.7546]],

        ...,

        [[-1.0134,  2.0086, -2.6476,  ...,  1.0799,  0.3678,  1.8758],
         [-0.2495, -1.5002, -1.99